In [387]:
%matplotlib inline
import pprint  
import math
import numpy as np
import numpy.random as npr
import pandas as pd
import random
from scipy import stats
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.extmath import cartesian
from bs4 import BeautifulSoup
import requests
import csv
import mechanize 
import time
import re
from collections import defaultdict

#r = requests.get('http://www.listingallcars.com/results/New')
#data = r.text
#soup = BeautifulSoup(data)
#for link in sou
#p.find_all('a')



In [388]:
#Last year season gameid http://www.espn.com/nba/game?gameId=400899695
#This year is from http://www.espn.com/nba/playbyplay?gameId=400900000
#To http://www.espn.com/nba/playbyplay?gameId=400900255
url = 'http://www.espn.com/nba/playbyplay?gameId='
pagestart = 400900100
pageend = 400900255
output = pd.DataFrame()

In [389]:
###Helper functino for make or miss
def toBinary(string):
    if (string == 'made'):
        return 1
    else:
        return 0 
    
def Open_Browser(url, page):
    br.open(url+str(page))
    response = br.response()
    soup = BeautifulSoup(response,"html.parser")
    return soup

def scrape(text_team):
    shot = []
    shootername =[]
    top = []
    left = []
    status = []
    shootingteam = []
    datatext = []
    threepointer = []
    assists = []
    quarter = []
    shooterid = []
    
    for x in text_team.find_all('li'):
        status_value = str(x.get('class')[0])
        shootingteam_value = str(x.get('data-homeaway'))
        datatext_value = str(x.get('data-text'))
        quarter_value = str(x.get('data-period'))
        shooterid_value = str(x.get('data-shooter'))
        text = ('{} {}'.format(x.get('id'), x.get('style').split(';')[-3:-1]))
        text =text.split()
        shot_value = (re.search(r'\d+', text[0][4:]).group())
        left_value = float((re.search(r'\d+', text[1][4:]).group()))/100
        top_value = float((re.search(r'\d+', text[2][4:]).group()))/100
        if ('three point' or 'three') in datatext_value:
            threepointer_value = 1
        else: 
            threepointer_value = 0
        if ('(' in datatext_value and ')' in datatext_value):
            assists_value = re.search(r'\((.*?)\)',datatext_value).group(1)
        else:
            assists_value = '0'
        
        if ('assists' or 'assist') in assists_value:
            assists_value = assists_value[0:len(assists_value)-8]
        else: 
            assists_value = '0'
        if datatext_value.find('makes') != -1:
            shootername_value = datatext_value[0:datatext_value.find('makes')-1]
        elif datatext_value.find('misses') != -1:
            shootername_value = datatext_value[0:datatext_value.find('misses')-1]
        elif ('blocks' in datatext_value) or ('block' in datatext_value):
            a = datatext_value.rindex('blocks')
            b = datatext_value.rindex("'")
            shootername_value = datatext_value[a+len('blocks')+1:b]
        while shootername_value[len(shootername_value)-1]==' ':
            shootername_value = shootername_value[0:len(shootername_value)-1]
            
        shootername.append(shootername_value)
        shootingteam.append(shootingteam_value)
        shot.append(shot_value)
        top.append(top_value)
        left.append(left_value)
        status.append(toBinary(status_value))
        datatext.append(datatext_value)
        threepointer.append(threepointer_value)
        assists.append(assists_value)
        quarter.append(quarter_value)
        shooterid.append(shooterid_value)
    
    shootername = np.array(shootername)
    shootingteam = np.array(shootingteam)
    shot = np.array(shot)
    top = np.array(top)
    left = np.array(left)
    status = np.array(status)
    datatext = np.array(datatext)
    threepointer = np.array(threepointer)
    assists = np.array(assists)
    shooterid = np.array(shooterid)
    quarter = np.array(quarter)

    return shootingteam, shot, top, left, status, datatext, threepointer,assists, shootername, quarter, shooterid

In [390]:
# Open_Browser(url, 400900084)

# #print soup.find_all('data-date')
# print soup.find('div',{'class' : 'cscore_date-time'}).get('data-date')[0:10]

# # for a in soup.find_all('div',{'class' : 'cscore_date-time'}):
# #     print a.get('data-date')
print pagestart


400900100


In [391]:
for page in range(pagestart,pageend):
    soup = Open_Browser(url, page)
    print page
    date = soup.find('div',{'class' : 'cscore_date-time'}).get('data-date')[0:10]
    teams =  soup.find_all('span',{'class' : 'cscore_name cscore_name--long'})
    home_team = teams[0].contents
    away_team = teams[1].contents

    text_home_team = soup.find('ul', { "class" : "shots home-team" })
    text_away_team = soup.find('ul', { "class" : "shots away-team" })

    shootingteam,shot,top,left,status, datatext, threepointer,assists, shootername,quarter, shooterid = scrape(text_home_team)
    HomeTeam = pd.DataFrame({'Date' : str(date)*len(shot),'Shooting Team': shootingteam,'Shooter Name' : shootername,'Shooter ID':shooterid,'Shot Number':shot,
                             'Quarter':quarter,'Left Position': left,'Top Position': top,'Home Team': home_team*(len(shot)),
                            'Away Team': away_team*len(shot),'Shot Status': status,'Three Pointer': threepointer,
                             'Assists': assists,'Text':datatext})

    shootingteam,shot,top,left,status,datatext,threepointer,assists, shootername,quarter, shooterid = scrape(text_away_team)
    AwayTeam = pd.DataFrame({'Date' : str(date)*len(shot),'Shooting Team': shootingteam,'Shooter Name' : shootername,'Shooter ID':shooterid,'Shot Number':shot,
                             'Quarter':quarter,'Left Position': left,'Top Position': top,'Home Team': home_team*(len(shot)),
                            'Away Team': away_team*len(shot),'Shot Status': status,'Three Pointer' : threepointer,
                             'Assists': assists,'Text':datatext})

    HomeTeam = HomeTeam[['Date','Shooting Team','Shooter Name','Shooter ID','Shot Number','Quarter','Left Position','Top Position','Home Team','Away Team',
                         'Shot Status','Three Pointer','Assists', 'Text']]
    AwayTeam = AwayTeam[['Date','Shooting Team','Shooter Name','Shooter ID','Shot Number','Quarter','Left Position','Top Position','Home Team','Away Team',
                         'Shot Status','Three Pointer','Assists','Text']]
    
    frames = [HomeTeam, AwayTeam]
    current_output = pd.concat(frames)
    output = output.append(current_output)
    print len(output)

400900100
170
400900101
347
400900102
515
400900103
678
400900104
855
400900105
1021
400900106
1191
400900107
1378
400900108
1552
400900109
1727
400900110
1911
400900111
2067
400900112
2229
400900113
2404
400900114
2569
400900115
2728
400900116
2907
400900117
3056
400900118
3232
400900119
3396
400900120
3582
400900121
3750
400900122
3928
400900123
4100
400900124
4258
400900125
4412
400900126
4580
400900127
4755
400900128
4929
400900129
5070
400900130
5241
400900131
5416
400900132
5663
400900133
5855
400900134
6025
400900135
6198
400900136
6356
400900137
6544
400900138
6714
400900139
6886
400900140
7055
400900141
7243
400900142
7422
400900143
7583
400900144
7737
400900145
7918
400900146
8105
400900147
8276
400900148
8447
400900149
8623
400900150
8811
400900151
8978
400900152
9150
400900153
9337
400900154
9535
400900155
9698
400900156
9874
400900157
10046
400900158
10208
400900159
10372
400900160
10539
400900161
10695
400900162
10861
400900163
11027
400900164
11199
400900165
11362
400900

ValueError: unichr() arg not in range(0x10000) (narrow Python build)

In [392]:
Open_Browser(url,pagestart)

<!DOCTYPE html>\n\n<html class="no-icon-fonts">\n<head>\n<meta content="text/html; charset=unicode-escape" http-equiv="content-type"/>\n<meta content="IE=edge,chrome=1" http-equiv="x-ua-compatible"/>\n<meta content="initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport">\n<link href="http://www.espn.com/nba/playbyplay?gameId=400900100" rel="canonical"/>\n<title>Timberwolves vs. Suns - Play-By-Play - January 24, 2017 - ESPN</title>\n<meta content="Get play-by-play updates on the Minnesota Timberwolves vs. Phoenix Suns basketball game." name="description"/>\n<meta content="" name="news_keywords"/>\n<meta content="" name="keywords"/>\n<meta content="116656161708917" property="fb:app_id"/>\n<meta content="ESPN.com" property="og:site_name"/>\n<meta content="http://www.espn.com/nba/playbyplay?gameId=400900100" property="og:url"/>\n<meta content="Timberwolves vs. Suns - Play-By-Play - January 24, 2017 - ESPN" property="og:title"/>\n<meta content="Get play-by-play updates on t

In [365]:
s = "Andre 'Drummond' b'locks 'Garrett Temple 's 2-foot  layup"

'blocks' in s
b = s.rindex("'")

print b

41


In [263]:
#############################Write Dataframe to .csv
output.to_csv('MarchMadnessScrape.csv', sep = ',', encoding = 'utf-8' , index = False)
